In [13]:
import os
import glob
import pandas as pd

In [14]:
pwd

'/Users/michaelpiacentino/Drive/git/data/SMPD3/LamininScores'

In [15]:
# Add analysis date here to apply to dataframe
analysis_date = '20181218'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Split values Image name columns
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], df['Somites'], df['ImageMag'], 
        df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Date,Experiment,Image,Target,Treatment,Dose,Embryo,Somites,Section,Control Side,Experimental Side,ExptDate,Stains,ImageMag
0,20180514,2018.05.14_SMPD3_and_Control_MOs,20180514_SMPD3MO_Pax7Laminin_Emb13_8ss_20x_sec...,SMPD3,SMPD3MO,0.8 mM,Emb13,8ss,sec3.tif,Complete channel,Complete channel,20180514,Pax7Laminin,20x
1,20180514,2018.05.14_SMPD3_and_Control_MOs,20180514_SMPD3MO_Pax7Laminin_Emb1_8ss_20x_sec3...,SMPD3,SMPD3MO,0.8 mM,Emb1,8ss,sec3.tif,Complete channel,Complete channel,20180514,Pax7Laminin,20x
2,20180514,2018.05.14_SMPD3_and_Control_MOs,20180514_SMPD3MO_Pax7Laminin_Emb1_8ss_20x_sec4...,SMPD3,SMPD3MO,0.8 mM,Emb1,8ss,sec4.tif,Complete channel,No channel,20180514,Pax7Laminin,20x
3,20180514,2018.05.14_SMPD3_and_Control_MOs,20180514_SMPD3MO_Pax7Laminin_Emb1_8ss_20x_sec5...,SMPD3,SMPD3MO,0.8 mM,Emb1,8ss,sec5.tif,Complete channel,Incomplete channel,20180514,Pax7Laminin,20x
4,20180514,2018.05.14_SMPD3_and_Control_MOs,20180514_SMPD3MO_Pax7Laminin_Emb3_9ss_20x_sec1...,SMPD3,SMPD3MO,0.8 mM,Emb3,9ss,sec1.tif,Complete channel,Incomplete channel,20180514,Pax7Laminin,20x


In [16]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = (full_df.groupby(['Treatment', 'Image'])
                  ['Image', 'Treatment', 'Control Side', 'Experimental Side']).sum()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    cntl_scores = pd.DataFrame(df_treatment['Control Side'].value_counts())
    expt_scores = pd.DataFrame(df_treatment['Experimental Side'].value_counts())
    results = pd.concat([cntl_scores, expt_scores], axis=1, sort=True)
    results.fillna(0, inplace=True)
    results = results.astype(int)
    results.to_csv(analysis_date + '_' + treatment + '_LamininResults.csv')